In [47]:
import importlib.machinery
import os

# Get the current working directory
cwd = os.getcwd()
print(cwd)
# Define the module name and full path

module_name = "utils"
function_name = "hello"
file_path = os.path.join(cwd, "Lesson_04", "utils", f"{module_name}.py")

# Load the module
try:
    loader = importlib.machinery.SourceFileLoader(module_name, file_path)
    module = loader.load_module()
    hello = getattr(module, function_name)
except:
    file_path = os.path.join(cwd, "utils", f"{module_name}.py")
    loader = importlib.machinery.SourceFileLoader(module_name, file_path)
    module = loader.load_module()
    hello = getattr(module, function_name)

hello("test complete...")
print(file_path)
simulate_bankruptcy = getattr(module, "simulate_bankruptcy")
create_dashboard = getattr(module, "create_dashboard")


/home/cooneycw/jupyter-workspace
test complete...
/home/cooneycw/jupyter-workspace/Lesson_04/utils/utils.py


In [48]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import VBox, HBox, Label, Output
from ipywidgets import interact, FloatSlider, IntSlider
import pandas as pd


# Run a single simulation with default parameters
bankruptcy_times = simulate_bankruptcy()
print(f"Average time to bankruptcy: {np.mean(bankruptcy_times):.2f} years")
print(f"Median time to bankruptcy: {np.median(bankruptcy_times):.2f} years")


Average time to bankruptcy: 5.35 years
Median time to bankruptcy: 5.00 years


In [49]:
# Cell 4: Create the interactive dashboard
create_dashboard()

In [50]:
# Cell 5: Create a custom interactive widget for parameter exploration
from ipywidgets import interact_manual, FloatSlider, IntSlider

@interact_manual(
    initial_capital=IntSlider(min=100000, max=5000000, step=100000, value=1000000, description='Initial $:'),
    claim_rate=FloatSlider(min=1, max=20, step=0.5, value=5, description='Claims/Year:'),
    avg_claim_size=FloatSlider(min=10000, max=200000, step=5000, value=50000, description='Avg Claim $:'),
    investment_return=FloatSlider(min=0.0, max=0.15, step=0.01, value=0.05, description='Return %:'),
    num_simulations=IntSlider(min=10, max=1000, step=10, value=100, description='Simulations:')
)
def run_custom_simulation(initial_capital, claim_rate, avg_claim_size, investment_return, num_simulations):
    # Run the simulation with custom parameters
    bankruptcy_times = simulate_bankruptcy(
        initial_capital=initial_capital,
        claim_rate=claim_rate,
        avg_claim_size=avg_claim_size,
        investment_return=investment_return,
        num_simulations=num_simulations
    )

    # Create visualization
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))

    # Create histogram with cumulative distribution
    n, bins, patches = ax.hist(bankruptcy_times, bins=30, alpha=0.7, edgecolor='black', label='Frequency')
    ax2 = ax.twinx()
    ax2.hist(bankruptcy_times, bins=30, cumulative=True, histtype='step',
             color='red', linewidth=2, label='Cumulative', density=True)

    ax.set_xlabel('Years until Bankruptcy')
    ax.set_ylabel('Frequency')
    ax2.set_ylabel('Cumulative Probability')
    ax.set_title(f'Bankruptcy Simulation Results\n(Mean: {np.mean(bankruptcy_times):.1f} years, Median: {np.median(bankruptcy_times):.1f} years)')

    # Add statistics text box
    stats_text = f"""Statistics:
Mean: {np.mean(bankruptcy_times):.1f} years
Median: {np.median(bankruptcy_times):.1f} years
Std Dev: {np.std(bankruptcy_times):.1f} years
Min: {np.min(bankruptcy_times):.1f} years
Max: {np.max(bankruptcy_times):.1f} years
Prob(survive 10y): {(bankruptcy_times > 10).mean():.2%}"""

    ax.text(0.72, 0.98, stats_text, transform=ax.transAxes,
            bbox=dict(boxstyle="round,pad=0.5", facecolor="wheat", alpha=0.8),
            verticalalignment='top', fontsize=10)

    ax.legend(loc='upper left')
    ax2.legend(loc='upper center')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


interactive(children=(IntSlider(value=1000000, description='Initial $:', max=5000000, min=100000, step=100000)…

In [51]:
# Cell 6: Create a parameter sensitivity analysis with dynamic sliders
def sensitivity_analysis():
    # Create output widget
    output = Output()

    # Define parameter ranges dictionary
    parameter_ranges = {
        'claim_rate': {
            'min': 1.0,
            'max': 20.0,
            'step': 0.5,
            'default_min': 2.0,
            'default_max': 10.0,
            'description': 'Claims/Year',
            'unit': 'claims/year'
        },
        'avg_claim_size': {
            'min': 10000,
            'max': 200000,
            'step': 5000,
            'default_min': 20000,
            'default_max': 100000,
            'description': 'Average Claim Size',
            'unit': '$'
        },
        'investment_return': {
            'min': 0.0,
            'max': 0.15,
            'step': 0.01,
            'default_min': 0.02,
            'default_max': 0.10,
            'description': 'Investment Return',
            'unit': '%'
        }
    }

    # Create widgets
    parameter_name = widgets.Dropdown(
        options=['claim_rate', 'avg_claim_size', 'investment_return'],
        value='claim_rate',
        description='Parameter:'
    )

    # Create sliders with initial values
    min_value = FloatSlider(description='Min Value:')
    max_value = FloatSlider(description='Max Value:')
    num_points = IntSlider(min=3, max=20, step=1, value=8, description='# Points:')

    # Create run button
    run_button = widgets.Button(description='Run Analysis')

    # Function to update slider ranges when parameter changes
    def update_slider_ranges(change):
        selected_param = change['new']
        param_info = parameter_ranges[selected_param]

        # Update min value slider
        min_value.min = param_info['min']
        min_value.max = param_info['max']
        min_value.step = param_info['step']
        min_value.value = param_info['default_min']
        min_value.description = f'Min {param_info["description"]}:'

        # Update max value slider
        max_value.min = param_info['min']
        max_value.max = param_info['max']
        max_value.step = param_info['step']
        max_value.value = param_info['default_max']
        max_value.description = f'Max {param_info["description"]}:'

    # Set up observer for parameter dropdown
    parameter_name.observe(update_slider_ranges, names='value')

    # Initialize sliders with values for the default parameter
    update_slider_ranges({'new': parameter_name.value})

    def run_sensitivity_analysis(button):
        parameter = parameter_name.value
        min_val = min_value.value
        max_val = max_value.value
        n_points = num_points.value

        with output:
            output.clear_output()

            # Generate parameter values
            param_values = np.linspace(min_val, max_val, n_points)
            results = []

            # Base parameters
            base_params = {
                'claim_rate': 5,
                'avg_claim_size': 50000,
                'investment_return': 0.05,
                'num_simulations': 100
            }

            # Run simulations for each parameter value
            for value in param_values:
                params = base_params.copy()
                params[parameter] = value

                bankruptcy_times = simulate_bankruptcy(**params)
                results.append({
                    'value': value,
                    'mean_bankruptcy_time': np.mean(bankruptcy_times),
                    'median_bankruptcy_time': np.median(bankruptcy_times),
                    'std_bankruptcy_time': np.std(bankruptcy_times)
                })

            df_results = pd.DataFrame(results)

            # Plot results
            fig, ax = plt.subplots(1, 1, figsize=(10, 6))

            # Adjust x-axis formatting based on parameter type
            if parameter == 'investment_return':
                ax.errorbar(df_results['value'] * 100, df_results['mean_bankruptcy_time'],
                           yerr=df_results['std_bankruptcy_time'], fmt='o-', capsize=5,
                           label='Mean ± Std Dev')
                ax.plot(df_results['value'] * 100, df_results['median_bankruptcy_time'],
                       's--', label='Median')
                ax.set_xlabel(f'{parameter_ranges[parameter]["description"]} (%)')
            else:
                ax.errorbar(df_results['value'], df_results['mean_bankruptcy_time'],
                           yerr=df_results['std_bankruptcy_time'], fmt='o-', capsize=5,
                           label='Mean ± Std Dev')
                ax.plot(df_results['value'], df_results['median_bankruptcy_time'],
                       's--', label='Median')
                if parameter == 'avg_claim_size':
                    ax.set_xlabel(f'{parameter_ranges[parameter]["description"]} ($)')
                    # Format x-axis labels as currency
                    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
                else:
                    ax.set_xlabel(f'{parameter_ranges[parameter]["description"]}')

            ax.set_ylabel('Years until Bankruptcy')
            ax.set_title(f'Sensitivity Analysis: {parameter_ranges[parameter]["description"]}')
            ax.legend()
            ax.grid(True, alpha=0.3)

            plt.tight_layout()
            plt.show()

            # Show data table with appropriate formatting
            if parameter == 'investment_return':
                df_results['value'] = df_results['value'] * 100  # Convert to percentage
                df_results = df_results.round(2)
                df_results = df_results.rename(columns={'value': 'Return (%)'})
            elif parameter == 'avg_claim_size':
                df_results['value'] = df_results['value'].apply(lambda x: f'${x:,.0f}')
                df_results = df_results.rename(columns={'value': 'Claim Size'})
            else:
                df_results = df_results.round(2)
                df_results = df_results.rename(columns={'value': 'Claims/Year'})

            display(df_results)

    # Connect button to function
    run_button.on_click(run_sensitivity_analysis)

    # Create layout
    controls = VBox([
        HBox([parameter_name]),
        HBox([min_value]),
        HBox([max_value]),
        HBox([num_points]),
        run_button
    ])

    # Display interface
    display(controls)
    display(output)

In [52]:
# Run the sensitivity analysis
sensitivity_analysis()

Output()